In [1]:
%%time
# endpoint invocation script
# expects the file test_point.csv to be located in the same diretory

#endpoint_name = 'XGBoostEP-2020-03-15-16-23-35'
#endpoint_name = 'XGBoostEP-2020-03-15-20-37-40'
endpoint_name = 'XGBoostEP-2020-03-18-20-15-31'

import os
import boto3
import re
import json
from sagemaker import get_execution_role
import sagemaker
import time

sess = sagemaker.Session()
region = boto3.Session().region_name
role = get_execution_role()
bucket=sess.default_bucket()

runtime_client = boto3.client('runtime.sagemaker')

file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
#print('Predicted Class Probabilities: {}.'.format(result))
print(response)

file_name = 'test_point.csv'
with open(file_name, 'r') as f:
    payload = f.read().strip()
    

{'ResponseMetadata': {'RequestId': '03c124b4-569f-438d-96d7-743dd8c212b1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '03c124b4-569f-438d-96d7-743dd8c212b1', 'x-amzn-invoked-production-variant': 'model-Blue', 'date': 'Wed, 18 Mar 2020 20:26:23 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '2'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'model-Blue', 'Body': <botocore.response.StreamingBody object at 0x7f83014a7a90>}
CPU times: user 960 ms, sys: 95.2 ms, total: 1.06 s
Wall time: 1.8 s


In [ ]:
N = 20000 #5000

for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    if i % (N/100) == 0:
        invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
        invocation_code = response['ResponseMetadata']['HTTPStatusCode']
        print ("target model: {}, return code {}; {} of {} invocations".format(invoked_model, invocation_code, i, N))
    #time.sleep(0.1)

In [ ]:
N = 200 #5000
print("started model invocation for {} iterations". format(N))
for i in range(N):
    response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
    invoked_model = response['ResponseMetadata']['HTTPHeaders']['x-amzn-invoked-production-variant']
    invocation_code = response['ResponseMetadata']['HTTPStatusCode']
    if (invocation_code != 200):
        print ("error invoking model {} ".format(invocation_code))